## Install Required Libraries

In [ ]:
!pip install langchain langchain-groq langchain_community crewai_tools python-dotenv ipywidgets google-search-results -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.7/552.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.3 

In [ ]:
!pip install --upgrade requests -q

## Library

In [ ]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_community.utilities import GoogleSerperAPIWrapper # Add this line
from langchain.agents import Tool # Add this line
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

## Define LangModel and Tools

Make sure to set these secrets in your Colab environment

- `GROQ_API_KEY`: Your API key from [Groq Cloud](console.groq.com)
- `SERPER_API_KEY`: Your API key from [Serper.dev](https://serper.dev)

In [ ]:
# Environment Variables
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # LangChain uses SERPER_API_KEY by default

print("Secrets loaded.")

Secrets loaded.


In [ ]:
# Initialize LLM
# You can change the model name if needed
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")
print("Groq LLM Initialized.")

Groq LLM Initialized.


In [ ]:
# Initialize Tools
search_wrapper = GoogleSerperAPIWrapper() # Instantiate LangChain's wrapper
search_tool = Tool( # Create a LangChain Tool
    name="Internet Search",
    func=search_wrapper.run,
    description="Useful for when you need to answer questions about current events, real-time information, or things you don't know."
)
tools = [search_tool] # Pass the LangChain Tool object
print("Tools Initialized.")

Tools Initialized.


## Define Agent

In [ ]:
# AI Learning Companion
agent_role_learning = "AI Learning Companion"
goal_learning = "Assist users in learning new subjects by explaining complex concepts in simple terms, finding relevant educational resources (articles, videos, courses), and helping create personalized study plans."
backstory_learning = """
You are 'PayoBelajarBot', a knowledgeable and patient AI tutor. You break down difficult topics, curate high-quality learning materials from the web, and adapt to the user's learning pace.
Your mission is to make learning accessible, engaging, and effective for everyone, regardless of their background knowledge.
You encourage curiosity and provide structured paths to mastering new skills.
"""
system_message_learning = f"Role: {agent_role_learning}\nGoal: {goal_learning}\nBackstory: {backstory_learning}"

In [ ]:
# Choose Agent Role
# Select which agent role you want to use by uncommenting one line:
chosen_system_message = system_message_learning

print(f"Using Agent Role:\n{chosen_system_message}\n")

Using Agent Role:
Role: AI Travel Agent
Goal: Plan personalized travel itineraries, find the best deals on flights and accommodations, and provide recommendations for destinations based on user preferences and budget.
Backstory: 
You are 'PegiMaenBot', an expert AI travel consultant with access to real-time travel data and trends.
You excel at understanding user needs, crafting unique travel experiences, and finding cost-effective solutions.
Your goal is to make travel planning seamless and exciting, offering insights into hidden gems and popular spots alike.
You always check for the latest travel advisories and local information.




In [ ]:
# Create Agent
# Get the ReAct prompt template
# You can explore other prompt templates from Langchain Hub: https://smith.langchain.com/hub
prompt_template = hub.pull("hwchase17/react")

# Add the specific role system message to the base prompt
prompt_template.template = chosen_system_message + "\n\n" + prompt_template.template

# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt_template)
print("Agent created.")

# Create the Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print("Agent Executor created.")

Agent created.
Agent Executor created.


## Define Task

In [ ]:
# Define Task
# Define a task relevant to the CHOSEN agent role above
# Example for Learning Companion:
task_description = "Explain the concept of 'quantum computing' in simple terms for a beginner and find a good introductory video on the topic."

print(f"\nTask defined:\n{task_description}")


Task defined:
Find me budget-friendly flights and a mid-range hotel (3-4 stars) for a 5-day trip from Jakarta (CGK) to Tokyo (NRT or HND) in the first week of December 2025. Also, suggest one unique cultural activity.


## Run The Agent

In [ ]:
print("Agent execution starting...")

# Run the agent with the defined task
result = agent_executor.invoke({"input": task_description})

print("\nAgent execution finished.")

# Print the final output
print("\n## Final Output:")
print(result['output'])

Agent execution starting...


> Entering new AgentExecutor chain...
To find budget-friendly flights and a mid-range hotel for a 5-day trip from Jakarta (CGK) to Tokyo (NRT or HND) in the first week of December 2025, and suggest a unique cultural activity, I should start by searching for available flights that fit the budget and time frame.

Action: Internet Search
Action Input: "budget flights from CGK to NRT or HND first week December 2025"Cheap Flights from Jakarta (CGK) to Tokyo (NRT) start at $151 for one-way and $345 for round trip. Earn your airline miles on top of our rewards! Missing: budget | Show results with:budget. Find flights to Tokyo Narita Airport from $158. Fly from Jakarta on EVA Air, Air China, China Eastern and more. Search for Tokyo Narita Airport flights on ... Missing: budget | Show results with:budget. Nonstop. from $696. Typical price: $590–1,100. $590. $1,100. 21 weekly nonstop flights. CGK - HND, NRTEvery day. MTWTFSS. Find flights from $696. JAL. Nonstop. Mi